# 1. Library Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


# 2. 데이터 불러오기

In [ ]:
df_transactions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KB/card_transaction2.csv')

card_benefit_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KB/card2.csv')

In [ ]:
df_transactions

In [ ]:
%matplotlib inline

import matplotlib as mpl  # 기본 설정
import matplotlib.pyplot as plt  # 그래프 그리기
import matplotlib.font_manager as fm  # 폰트 관리


!apt-get update -qq         # apt-get 패키지 설치 명령어, -qq : 에러외 메세지 숨기기
!apt-get install fonts-nanum* -qq #나눔글꼴 설치

fe = fm.FontEntry(fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) #폰트설정



# 3. 데이터 확인 및 전처리

In [ ]:
df_transactions.head()

In [ ]:
df_transactions['날짜'] = pd.to_datetime(df_transactions['날짜'])


In [ ]:
card_benefit = card_benefit_df.groupby(['카드명','혜택 카테고리']).mean().reset_index()


In [ ]:
card_benefit.drop(card_benefit.loc[card_benefit['혜택 카테고리'].isin(['음식점','편의점'])].index,inplace=True)

In [ ]:
card_benefit

# 4. 고객, 카드, 혜택 카테고리별 Count

In [ ]:
# 카테고리별 이용 횟수를 계산하기 위한 데이터프레임 생성
df_counts = pd.DataFrame(columns=['고객번호', '카드명', '혜택 카테고리', '이용 횟수','CREDIT분류'])

# 각 고객에 대해 혜택 카테고리별 이용 횟수 계산
rows_to_add = []
for idx, row in df_transactions.iterrows():
    고객번호 = row['고객번호']
    for benefit_idx, benefit_row in card_benefit.iterrows():
        카드명 = benefit_row['카드명']
        혜택_카테고리 = benefit_row['혜택 카테고리']
        신용카드분류 = benefit_row['Credit']
        if row.get(혜택_카테고리, 0) > 0:
            rows_to_add.append({
                '고객번호': 고객번호,
                '카드명': 카드명,
                'CREDIT분류': 신용카드분류,
                '혜택 카테고리': 혜택_카테고리,
                '이용 횟수': 1
            })

# DataFrame으로 변환 후 concat
df_counts = pd.concat([df_counts, pd.DataFrame(rows_to_add)], ignore_index=True)

# 결과 출력
df_counts

In [ ]:
df_counts.groupby(['고객번호','카드명','혜택 카테고리','CREDIT분류']).sum()


In [ ]:
grouped_df = df_counts.groupby(['고객번호','카드명','혜택 카테고리','CREDIT분류']).sum().reset_index()

grouped_df.columns = ['고객번호', '카드명', '혜택 카테고리','CREDIT분류', '이용 횟수']


grouped_df

In [ ]:
grouped_df.info()

In [ ]:
grouped_df['이용 횟수'] = pd.to_numeric(grouped_df['이용 횟수'])

In [ ]:
grouped_df.loc[(grouped_df['고객번호']==1)].groupby(['고객번호','카드명','혜택 카테고리','CREDIT분류']).sum().sort_values('이용 횟수',ascending=False)

In [ ]:
grouped_df.loc[(grouped_df['고객번호']==1)].groupby(['고객번호','카드명','혜택 카테고리','CREDIT분류']).sum().nlargest(4,'이용 횟수')


# 5.카드별 혜택 총합 수

In [ ]:
# 피벗 테이블 생성
pivot_df = grouped_df.pivot_table(index=['고객번호','카드명','CREDIT분류'],columns='혜택 카테고리',values='이용 횟수',fill_value=0)

# 컬럼 이름 정렬 (카테고리 순서 유지)
pivot_df = pivot_df.sort_index(axis=1)

pivot_df

In [ ]:
pivot_df.loc[2]

In [ ]:
row_sums = pivot_df.sum(axis=1)
# 총합을 데이터프레임으로 변환
row_sums_df = row_sums.reset_index(name='총합')

# 고객번호와 총합으로 정렬
sorted_row_sums = row_sums_df.sort_values(by=['고객번호', '총합'], ascending=[False, False])

print(sorted_row_sums)

sum_df = sorted_row_sums.sort_values(['고객번호','총합'],ascending=[True,False])


In [ ]:
sum_df = sorted_row_sums.sort_values(['고객번호','총합'],ascending=[True,False])


In [ ]:
sum_df.info()

## 5.1. **다담카드** 총합과 선택형 혜택 더하기  
다담카드는 기본혜택 + α(선택형 혜택)이 나뉘어지기 때문에 기본혜택을 각각 선택혜택에 더해야 함

In [ ]:
array_1= sum_df.loc[sum_df['카드명']=='다담'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values + sum_df.loc[sum_df['카드명']=='다담_생활'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values
dd1 = array_1.tolist()
sum_df.loc[sum_df['카드명']=='다담_생활','총합'] = dd1

In [ ]:
array_1= sum_df.loc[sum_df['카드명']=='다담'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values[1] + sum_df.loc[sum_df['카드명']=='다담_직장인'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values
dd1 = array_1.tolist()
sum_df.loc[sum_df['카드명']=='다담_직장인','총합'] = dd1

In [ ]:
array_1= sum_df.loc[sum_df['카드명']=='다담'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values[1] + sum_df.loc[sum_df['카드명']=='다담_쇼핑'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values
dd1 = array_1.tolist()
sum_df.loc[sum_df['카드명']=='다담_쇼핑','총합'] = dd1

In [ ]:
array_1= sum_df.loc[sum_df['카드명']=='다담'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values + sum_df.loc[sum_df['카드명']=='다담_레저'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values
dd1 = array_1.tolist()
sum_df.loc[sum_df['카드명']=='다담_레저','총합'] = dd1

In [ ]:
array_1= sum_df.loc[sum_df['카드명']=='다담'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values[1] + sum_df.loc[sum_df['카드명']=='다담_교육'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values
dd1 = array_1.tolist()
sum_df.loc[sum_df['카드명']=='다담_교육','총합'] = dd1

In [ ]:
sum_df.loc[sum_df['카드명']=='다담'].groupby(['고객번호','카드명','CREDIT분류']).sum()['총합'].values

In [ ]:
sum_df = sum_df.loc[sum_df['카드명'] != '다담']

In [ ]:
sum_df.loc[sum_df['고객번호']==40]


In [ ]:
sum_df.loc[sum_df['고객번호']==40]


In [ ]:
sum_df.loc[sum_df['고객번호']==2].sort_values('총합',ascending=False)
